# TruBot

Example setup and monitoring of a conversational bot with context made up of the TruEra website.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
from pprint import PrettyPrinter
from typing import Sequence

from IPython.display import JSON
# imports from langchain to build app
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import PromptTemplate
from langchain.vectorstores import DocArrayHnswSearch
import numpy as np

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import Huggingface
from trulens_eval import Query
from trulens_eval import Tru
from trulens_eval import tru_feedback
from trulens_eval import TruChain
from trulens_eval.keys import *
from trulens_eval.tru_db import Query
from trulens_eval.tru_db import Record
from trulens_eval.tru_feedback import Feedback
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import Step
from trulens_eval.utils.langchain import WithFeedbackFilterDocuments

# if using Pinecone vectordb:
# from langchain.vectorstores import Pinecone
# import pinecone

tru = Tru()
pp = PrettyPrinter()

2023-06-03 22:23:21,194 - numexpr.utils - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-06-03 22:23:21,197 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY
2023-06-03 22:23:21,541 - trulens_eval.util - DEBUG - *** Creating new Tru singleton instance for name = None ***


/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "gpt-3.5-turbo"
chain_id = "TruBot"

# Pinecone configuration if using pinecone.
# pinecone.init(
#    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#    environment=PINECONE_ENV  # next to api key in console
#)
#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.chain.combine_docs_chain._call.args.inputs.
    input_documents[:].page_content
).aggregate(np.min)

def new_conversation(
    lang_prompt_fix: bool = False,
    context_prompt_fix: bool = False,
    context_filter_fix: bool = False,
    feedbacks: Sequence[Feedback] = None
):
    """
    Create a chain for a new conversation (blank memory). Set flags to enable
    adjustments to prompts or add context filtering.
    """
    
    assert not(lang_prompt_fix and context_prompt_fix), "Cannot use both prompt fixes at the same time."

    # Embedding needed for Pinecone vector db.
    embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    # Pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
    docsearch = DocArrayHnswSearch.from_params(
        embedding=embedding,
        work_dir='hnswlib_trubot',
        n_dim=1536,
        max_elements=1024
    )
    retriever = docsearch.as_retriever()

    # Better contexts fix, filter contexts with relevance:
    if context_filter_fix: 
        retriever = WithFeedbackFilterDocuments.of_retriever(
            retriever=retriever, feedback=f_qs_relevance, threshold = 0.5
        )

    # Conversational chain puts it all together.
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Need to copy these otherwise various chains will feature templates that
    # point to the same objects.
    chain.combine_docs_chain.llm_chain.prompt = \
        chain.combine_docs_chain.llm_chain.prompt.copy()
    chain.combine_docs_chain.document_prompt = \
        chain.combine_docs_chain.document_prompt.copy()

    # Language mismatch fix:
    if lang_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use the following pieces of context to answer the question at the end " \
            "in the same language as the question. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "

    # Poor contexts fix using prompts:
    elif context_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use only the relevant contexts to answer the question at the end " \
            ". Some pieces of context may not be relevant. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "Contexts: \n{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "
        chain.combine_docs_chain.document_prompt.template = "\tContext: {page_content}"

    # Trulens instrumentation.
    tc = Tru().Chain(chain=chain, feedbacks=feedbacks, verbose=True)

    return tc

2023-06-03 22:23:21,612 - trulens_eval.util - DEBUG - *** Creating new Endpoint singleton instance for name = huggingface ***
2023-06-03 22:23:21,614 - trulens_eval.provider_apis - DEBUG - *** Creating huggingface endpoint ***


huggingface api: 0requests [00:00, ?requests/s]

2023-06-03 22:23:21,632 - trulens_eval.util - DEBUG - *** Creating new Endpoint singleton instance for name = openai ***
2023-06-03 22:23:21,632 - trulens_eval.provider_apis - DEBUG - *** Creating openai endpoint ***


In [4]:
# Instantiate a chain

tc = new_conversation(
    #feedbacks=[f_lang_match],
    context_filter_fix=True
)

INFO - docarray - DB config created


2023-06-03 22:23:22,436 - docarray - INFO - DB config created


INFO - docarray - Runtime config created


2023-06-03 22:23:22,437 - docarray - INFO - Runtime config created


DEBUG - docarray - Working directory set to hnswlib_trubot


...

2023-06-03 22:23:22,450 - docarray - DEBUG - DB path set to hnswlib_trubot/docs_sqlite.db


INFO - docarray - Connection to DB has been established


2023-06-03 22:23:22,451 - docarray - INFO - Connection to DB has been established


INFO - docarray - HnswDocumentIndex[DocArrayDoc] has been initialized


2023-06-03 22:23:22,452 - docarray - INFO - HnswDocumentIndex[DocArrayDoc] has been initialized
2023-06-03 22:23:22,453 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<class 'openai.api_resources.com' of type '<class 'type'>'.
2023-06-03 22:23:22,454 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<class 'langchain.schema.SystemM' of type '<class 'pydantic.main.ModelMetaclass'>'.
2023-06-03 22:23:22,455 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<class 'openai.api_resources.com' of type '<class 'type'>'.
2023-06-03 22:23:22,455 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<class 'openai.api_resources.com' of type '<class 'type'>'.
2023-06-03 22:23:22,455 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<function new_conversation.<loca' of type '<class 'function'>'.
2023-06-03 22:23:22,456 - trulens_eval.util - DEBUG - Don't know how to jsonify an object '<langchain.vectorstores.docarr

In [5]:
# Call the chain

res, record = tc.call_with_record("Who is Shayak?")

2023-06-03 22:23:22,645 - trulens_eval.tru_chain - DEBUG - Calling instrumented method <function BaseConversationalRetrievalChain._call at 0x11eb7d9d0> on *.chain
2023-06-03 22:23:22,919 - trulens_eval.util - DEBUG - *** Creating new TP singleton instance for name = None ***
2023-06-03 22:23:22,982 - trulens_eval.util - DEBUG - looking find_call_with_record, pass : FrameInfo(frame=<frame at 0x16b39bb00, file '/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/langchain/chains/base.py', line 136, code __call__>, filename='/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/langchain/chains/base.py', lineno=136, function='__call__', code_context=['                else self._call(inputs)\n'], index=0)
2023-06-03 22:23:22,983 - trulens_eval.util - DEBUG - looking find_call_with_record found: FrameInfo(frame=<frame at 0x11ea67860, file '/Users/piotrm/repos/trulens/trulens_eval/trulens_eval/tru_chain.py', line 222, code call_wi

DEBUG - docarray - Executing `find` for search field embedding


2023-06-03 22:23:23,590 - docarray - DEBUG - Executing `find` for search field embedding
2023-06-03 22:23:23,600 - trulens_eval.tru_chain - DEBUG - Calling instrumented method <function Feedback.__call__ at 0x13f75fc10> on *.chain.retriever.feedback
2023-06-03 22:23:23,602 - trulens_eval.util - DEBUG - looking find_call_with_record, pass : FrameInfo(frame=<frame at 0x16c82fc80, file '/Users/piotrm/repos/trulens/trulens_eval/trulens_eval/utils/langchain.py', line 38, code <lambda>>, filename='/Users/piotrm/repos/trulens/trulens_eval/trulens_eval/utils/langchain.py', lineno=38, function='<lambda>', code_context=['            lambda doc, query: self.feedback(query, doc.page_content) > self.threshold,\n'], index=0)
2023-06-03 22:23:23,602 - trulens_eval.util - DEBUG - Found thread starter frame. Will walk over frames in prior to thread start.
2023-06-03 22:23:23,602 - trulens_eval.util - DEBUG - looking find_call_with_record, pass : FrameInfo(frame=<frame at 0x16a7c87d0, file '/opt/homebre

2023-06-03 22:23:23,611 - openai - DEBUG - message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2023-06-03 22:23:23,611 - openai - DEBUG - api_version=None data='{"model": "gpt-3.5-turbo", "temperature": 0.0, "messages": [{"role": "system", "content": "You are a RELEVANCE classifier; providing the relevance of the given STATEMENT to the given QUESTION.\\nRespond only as a number from 1 to 10 where 1 is the least relevant and 10 is the most relevant.\\nNever elaborate.\\n\\nQUESTION: Who is Shayak?\\n\\nSTATEMENT: When Shayak started building production grade machine learning models for algorithmic trading 10 years ago, he realized the need for putting the \\u2018science\\u2019 back in \\u2018data science\\u2019. Since then, he has been building systems and leading research to make machine learning and big data systems more explainable, privacy compliant, and fair. Shayak\\u2019s research at Carnegie Mellon University introduced a number of pioneer

2023-06-03 22:23:24,209 - openai - DEBUG - message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2023-06-03 22:23:24,210 - openai - DEBUG - api_version=None data='{"model": "gpt-3.5-turbo", "temperature": 0.0, "messages": [{"role": "system", "content": "You are a RELEVANCE classifier; providing the relevance of the given STATEMENT to the given QUESTION.\\nRespond only as a number from 1 to 10 where 1 is the least relevant and 10 is the most relevant.\\nNever elaborate.\\n\\nQUESTION: Who is Shayak?\\n\\nSTATEMENT: When Shayak started building production grade machine learning models for algorithmic trading 10 years ago, he realized the need for putting the \\u2018science\\u2019 back in \\u2018data science\\u2019. Since then, he has been building systems and leading research to make machine learning and big data systems more explainable, privacy compliant, and fair. Shayak\\u2019s research at Carnegie Mellon University introduced a number of pioneer

2023-06-03 22:23:24,868 - openai - DEBUG - message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2023-06-03 22:23:24,868 - openai - DEBUG - api_version=None data='{"model": "gpt-3.5-turbo", "temperature": 0.0, "messages": [{"role": "system", "content": "You are a RELEVANCE classifier; providing the relevance of the given STATEMENT to the given QUESTION.\\nRespond only as a number from 1 to 10 where 1 is the least relevant and 10 is the most relevant.\\nNever elaborate.\\n\\nQUESTION: Who is Shayak?\\n\\nSTATEMENT: Anupam is passionate about enabling responsible adoption of artificial intelligence. As a Professor of Electrical & Computer Engineering and Computer Science at Carnegie Mellon University for over a decade, he has led groundbreaking research in the areas of AI explainability and governance as well as privacy and data protection. Anupam obtained PhD and MS degrees from Stanford University and a BTech from the Indian Institute of Technology

2023-06-03 22:23:25,855 - openai - DEBUG - message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2023-06-03 22:23:25,855 - openai - DEBUG - api_version=None data='{"model": "gpt-3.5-turbo", "temperature": 0.0, "messages": [{"role": "system", "content": "You are a RELEVANCE classifier; providing the relevance of the given STATEMENT to the given QUESTION.\\nRespond only as a number from 1 to 10 where 1 is the least relevant and 10 is the most relevant.\\nNever elaborate.\\n\\nQUESTION: Who is Shayak?\\n\\nSTATEMENT: Most recently, Shameek was Group Chief Data Officer at Standard Chartered Bank, where he helped the bank explore and adopt AI in multiple areas (e.g., credit, financial crime compliance, customer analytics, surveillance), and shaped the bank\\u2019s internal approach to responsible AI\\n\\nRELEVANCE: "}]}' message='Post details'
2023-06-03 22:23:25,855 - urllib3.util.retry - DEBUG - Converted retries value: 2 -> Retry(total=2, connect=Non

In [ ]:
res

In [ ]:
record.dict()

In [ ]:
for call in record.calls:
    print(" -> ".join(map(lambda f: str(f.path), call.chain_stack)))
    #for f in call.chain_stack:
    ## print(call.chain_stack)
    #print()

In [ ]:
dir(tc.chain.retriever.feedback.__call__)